### 0924 과제

In [10]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchmetrics.classification import F1Score, MulticlassF1Score
from torchinfo import summary

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [11]:
print(f"torch v.{torch.__version__}")
print(f"Pandas v.{pd.__version__}")

torch v.2.4.1
Pandas v.2.0.3


In [12]:
# Data Loading
DATA_PATH = "../data/"

TRAINFILE = r"C:\Hwan\openCV(TORCH_IMAGE)\data\mnist__test.csv"
TESTFILE = r"C:\Hwan\openCV(TORCH_IMAGE)\data\mnist__test.csv"

# CSV => DataFrame
TRAINDF = pd.read_csv(TRAINFILE)
TESTDF = pd.read_csv(TESTFILE)

# 확인
print(TRAINDF.head(1), TESTDF.head(1), sep ="\n")


   0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.659  0.660  0.661  \
0  0    0    0    0    0    0    0    0    0    0  ...      0      0      0   

   0.662  0.663  0.664  0.665  0.666  0.667  7  
0      0      0      0      0      0      0  2  

[1 rows x 785 columns]
   0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.659  0.660  0.661  \
0  0    0    0    0    0    0    0    0    0    0  ...      0      0      0   

   0.662  0.663  0.664  0.665  0.666  0.667  7  
0      0      0      0      0      0      0  2  

[1 rows x 785 columns]


[Model Class 설계 및 정의]

In [13]:
class MNISTMCFModel(nn.Module) :

    def __init__(self) :
        super().__init__()

        self.in_layer = nn.Linear(784, 1000)
        self.hd1_layer = nn.Linear(1000, 2000)
        self.hd2_layer = nn.Linear(2000, 3000)
        self.hd3_layer = nn.Linear(3000, 2000)
        self.hd4_layer = nn.Linear(2000, 1000)
        self.out_layer = nn.Linear(1000, 10)   

    # 순방향 학습 진행 메서드
    def forward(self, input_data) :

        # 입력층
        y = self.in_layer(input_data)   
        y = F.relu(y)   

        # 은닉층 
        y = self.hd1_layer(y)           
        y = F.relu(y) 
        y = self.hd2_layer(y) 
        y = F.relu(y) 
        y = self.hd3_layer(y) 
        y = F.relu(y) 
        y = self.hd4_layer(y)
        y = F.relu(y) 

        return self.out_layer(y)


In [14]:
model = MNISTMCFModel()

print(model)

MNISTMCFModel(
  (in_layer): Linear(in_features=784, out_features=1000, bias=True)
  (hd1_layer): Linear(in_features=1000, out_features=2000, bias=True)
  (hd2_layer): Linear(in_features=2000, out_features=3000, bias=True)
  (hd3_layer): Linear(in_features=3000, out_features=2000, bias=True)
  (hd4_layer): Linear(in_features=2000, out_features=1000, bias=True)
  (out_layer): Linear(in_features=1000, out_features=10, bias=True)
)


[DataSet Class 설계 및 정의]

In [15]:
class MNISTDataset(Dataset) :

    def __init__(self, featureDF, targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows

    def __getitem__(self, index) :
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [16]:
# DataFrame에서 feature, target 추출
featureDF = TRAINDF[TRAINDF.columns[:-1]] # 2차원 (150, 3)
targetDF = TRAINDF[TRAINDF.columns[-1:]]  # 2차원 (150, 1)

# 커스텀 데이터셋 인스턴스 생성
MNISTDS = MNISTDataset(featureDF, targetDF)

# DataLoader 인스턴스 생성
MNISTDL = DataLoader(MNISTDS)

for feature, label in MNISTDL :
    print(feature.shape, label.shape, feature, label)
    break

print(featureDF.shape, targetDF.shape)
print(featureDF)
print(targetDF)

torch.Size([1, 784]) torch.Size([1, 1]) tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 

[학습준비]

In [17]:
# 학습진행 관련 설정값
EPOCH = 250
BATCH_SIZE = 10
BATCH_CNT = TRAINDF.shape[0]//BATCH_SIZE
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 0.001 

In [18]:
# 학습용, 검증용, 테스트용 데이터 분리

X_train, X_val, y_train, y_val = train_test_split(featureDF, targetDF, random_state=1)
print(f"{X_train.shape} {X_val.shape}")
print(f"{y_train.shape} {y_val.shape}")
print(f"{type(X_train)} {type(y_val)}")

# 학습용, 검증용, 테스트용 데이터셋 생성
trainDS = MNISTDataset(X_train, y_train)
valDS = MNISTDataset(X_val, y_val)


# 학습용 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE) 

(7499, 784) (2500, 784)
(7499, 1) (2500, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [19]:
# 최적화
optimizer = optim.Adam(model.parameters(), lr =LR)

crossLoss = nn.CrossEntropyLoss()

[학습]

In [20]:
## 모델 파일관련
import os 

# 저장 경로
SAVE_PATH = '../models/work/MUL/'

#  저장 파일명
SAVE_FILE=SAVE_PATH+'model_train_wb.pth'

# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL = SAVE_PATH+'model_all.pth'

In [21]:
# 경로상 폴더 존재여부 체크 
if not os.path.exists(SAVE_PATH) :
    os.makedirs(SAVE_PATH) 

In [22]:
import schedule
import torch.optim.lr_scheduler as scheduler

In [23]:
# for a,b in trainDL:
#     print(a, type(a))

In [24]:
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[],[]]
CNT = len(trainDL)
print(f"BATCH_CNT = {CNT}")

# featureTS = torch.FloatTensor(featureDF.iloc[:-1].values)
# targetTS = torch.FloatTensor(targetDF.iloc[-1:].values)
BREAK_CNT = 0
LIMIT_VALUE = 5

for epoch in range(EPOCH):
    loss_total, score_total = 0, 0
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    for featureTS, targetTS in trainDL :

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답(=타겟)은 0차원 or 1차원, type = long
        loss = crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능평가 계산
        score = MulticlassF1Score(num_classes=10)(pre_y, targetTS.reshape(-1)) # 차원 맞춰주기 
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad() :
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val = model(val_featureTS)
        # 손실
        loss_val = crossLoss(pre_val, val_targetTS.reshape(-1).long())
        # 성능평가
        score_val = MulticlassF1Score(num_classes=10)(pre_val, val_targetTS.reshape(-1))
    
    # 에포크당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    print(f' [{epoch}/{EPOCH}]\n [TRAIN] LOSS : {LOSS_HISTORY[0][-1]}, SCORE : {SCORE_HISTORY[0][-1]}')
    print(f' [VALID] LOSS : {LOSS_HISTORY[1][-1]}, SCORE : {SCORE_HISTORY[1][-1]}')

if len(SCORE_HISTORY[1]) >= 2 :
    if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2] :
        BREAK_CNT += 1


# 손실감소(= 성능개선)가 안되는 경우 조기 종료
if BREAK_CNT > LIMIT_VALUE :
    print(f"{epoch}만큼 반복을 했는데 {scheduler.patience}EPOCH 성능개선이 없어서 조기종료함 !")
    break


    # # 성능이 좋은 학습 가중치 저장 
    # if len(SCORE_HISTORY[1]) == 1 :

    #     # 첫번째 모델이라 무조건 저장 
    #     # 모델 파라미터
    #     torch.save(model.state_dict(), SAVE_FILE)

    #     # 모델 정체 저장
    #     torch.save(model, SAVE_MODEL)

    # else :
    #     # 두번째부터는 첫번쩨 보다 다음 점수가 더 좋으면 (=성능이 좋으면) 저장
    #     if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]) :     
            
    #         torch.save(model.state_dict(), SAVE_FILE)
    #         torch.save(model, SAVE_MODEL)

BATCH_CNT = 750
 [0/250]
 [TRAIN] LOSS : 2.404666373252869, SCORE : 0.027277498272558053
 [VALID] LOSS : 2.300791025161743, SCORE : 0.021045096218585968
 [1/250]
 [TRAIN] LOSS : 2.3019667285283405, SCORE : 0.02917430895815293
 [VALID] LOSS : 2.301002025604248, SCORE : 0.021045096218585968
 [2/250]
 [TRAIN] LOSS : 2.3017146428426107, SCORE : 0.029764785153170426
 [VALID] LOSS : 2.301159381866455, SCORE : 0.021045096218585968
 [3/250]
 [TRAIN] LOSS : 2.3015638780593872, SCORE : 0.029764785153170426
 [VALID] LOSS : 2.30122971534729, SCORE : 0.021045096218585968
 [4/250]
 [TRAIN] LOSS : 2.3014682814280194, SCORE : 0.029764785153170426
 [VALID] LOSS : 2.3013100624084473, SCORE : 0.021045096218585968
 [5/250]
 [TRAIN] LOSS : 2.3013825333913167, SCORE : 0.029764785153170426
 [VALID] LOSS : 2.3013737201690674, SCORE : 0.021045096218585968
 [6/250]
 [TRAIN] LOSS : 2.3013067448933917, SCORE : 0.029764785153170426
 [VALID] LOSS : 2.3014419078826904, SCORE : 0.021045096218585968
 [7/250]
 [TRAIN] 

SyntaxError: 'break' outside loop (2993231158.py, line 66)

[TEST]

In [25]:
TESTFeatureDF = TESTDF[TESTDF.columns[:-1]]
TESTTargetDF = TESTDF[TESTDF.columns[-1:]]

In [26]:
TESTDS = Dataset(TESTFeatureDF, TESTTargetDF)
TESTDL = DataLoader(TESTDS, batch_size = BATCH_SIZE)

TypeError: object.__new__() takes exactly one argument (the type to instantiate)

In [27]:
# TEST
model.eval()

with torch.no_grad() :
    # 검증 데이터셋
    test_featureTS = torch.FloatTensor(TESTDS.featureDF.values)
    test_targetTS = torch.FloatTensor(TESTDS.targetDF.values)
    # 평가
    pre_val = model(test_featureTS)
    # 손실
    loss_val = crossLoss(pre_val, test_targetTS.reshape(-1).long())
    # 성능평가
    score_val = MulticlassF1Score(num_classes=10)(pre_val, test_targetTS.reshape(-1))
    print(f"Loss score {loss_val}  Score {pre_val}")

NameError: name 'TESTDS' is not defined